In [2]:
!pip install ultralytics roboflow

from ultralytics import YOLO
import torch

print("Torch version:", torch.__version__)

  Using cached ultralytics-8.3.203-py3-none-any.whl.metadata (37 kB)
  Using cached roboflow-1.2.9-py3-none-any.whl.metadata (9.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
Creating new Ultralytics Settin

In [5]:
from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/dataset"

Mounted at /content/drive


In [6]:
yaml_config = """
path: {project_path}
train: images/train
val: images/val

nc: 1
names: ["redwood"]
""".format(project_path=project_path)

with open("config.yaml", "w") as f:
    f.write(yaml_config)

In [16]:
!pip install albumentations==1.3.1 --quiet
import cv2, os
import albumentations as A
import numpy as np

img_dir = project_path + "/images/train"
label_dir = project_path + "/labels/train"

# Compose transformations
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.2,
                       rotate_limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.0, min_area=1e-6))

num_aug = 5

def clip_yolo_bbox(bbox):
    x, y, w, h = bbox
    x_min = max(0.0, x - w/2)
    y_min = max(0.0, y - h/2)
    x_max = min(1.0, x + w/2)
    y_max = min(1.0, y + h/2)
    x_new = (x_min + x_max) / 2
    y_new = (y_min + y_max) / 2
    w_new = x_max - x_min
    h_new = y_max - y_min
    return [x_new, y_new, w_new, h_new]

def filter_valid_bboxes(bboxes, classes, min_size=1e-6):
    valid_bboxes = []
    valid_classes = []
    for bbox, cls in zip(bboxes, classes):
        _, _, w, h = bbox
        if w > min_size and h > min_size:
            valid_bboxes.append(bbox)
            valid_classes.append(cls)
    return valid_bboxes, valid_classes

for img_file in os.listdir(img_dir):
    if not img_file.endswith(".jpg"):
        continue

    img_path = os.path.join(img_dir, img_file)
    label_path = os.path.join(label_dir, img_file.replace(".jpg", ".txt"))

    if not os.path.exists(label_path):
        print(f"⚠️ Skipping {img_file}, no label found.")
        continue

    image = cv2.imread(img_path)

    bboxes, class_labels = [], []
    with open(label_path) as f:
        for line in f:
            cls, x, y, bw, bh = map(float, line.split())
            bboxes.append([x, y, bw, bh])
            class_labels.append(int(cls))

    # Clip bboxes BEFORE augmentation
    bboxes = [clip_yolo_bbox(b) for b in bboxes]

    for i in range(num_aug):
        try:
            augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        except ValueError:
            # Skip augmentation if transform produces invalid boxes
            continue

        aug_img = augmented['image']
        aug_bboxes, aug_classes = filter_valid_bboxes(augmented['bboxes'], augmented['class_labels'])

        if len(aug_bboxes) == 0:
            continue

        aug_img_name = img_file.replace(".jpg", f"_aug{i}.jpg")
        cv2.imwrite(os.path.join(img_dir, aug_img_name), aug_img)

        aug_label_name = aug_img_name.replace(".jpg", ".txt")
        with open(os.path.join(label_dir, aug_label_name), "w") as f:
            for cls, (x, y, bw, bh) in zip(aug_classes, aug_bboxes):
                f.write(f"{cls} {x} {y} {bw} {bh}\n")

print("✅ Augmentation complete! Augmented files saved directly into train folders.")

✅ Augmentation complete! Augmented files saved directly into train folders.


In [8]:
model = YOLO("yolov8n.pt")
model.train(
    data="config.yaml",
    epochs=20,
    imgsz=416,
    batch=16,
    project="redwood_detection",
    workers=2,
    cache=True,
    name="yolov8n_redwood"
)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=config.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_redwood2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ec409f16cc0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [9]:
model = YOLO("/content/redwood_detection/yolov8n_redwood8/weights/best.pt")
results = model.predict(source="/content/drive/MyDrive/dataset/images/train", imgsz=640, conf=0.25)


FileNotFoundError: [Errno 2] No such file or directory: '/content/redwood_detection/yolov8n_redwood8/weights/best.pt'

In [10]:
metrics = model.val()
print(metrics)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.3 ms, read: 91.4±26.6 MB/s, size: 835.3 KB)
val: Scanning /content/drive/MyDrive/dataset/labels/val.cache... 34 images, 6 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 30.6Kit/s 0.0s
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.
val: Caching images (0.0GB RAM): 100% ━━━━━━━━━━━━ 40/40 17.4it/s 2.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.2it/s 14.0s
                   all         40      36193      0.255     0.0844      0.147     0.0421
Speed: 1.7ms preprocess, 214.1ms inference, 0.0ms loss, 28.1ms postprocess per image
Results saved to /content/redwood_detection/yolov8n_redwood22
ultralytics.utils.metrics.D

In [11]:
results = model.predict(source=f"{project_path}/images/val", save=True, conf=0.25)


image 1/40 /content/drive/MyDrive/dataset/images/val/Copy of redwood_image_05_aug4.jpg: 256x416 156 redwoods, 344.5ms
image 2/40 /content/drive/MyDrive/dataset/images/val/Copy of redwood_image_05_aug4_aug1.jpg: 256x416 97 redwoods, 281.5ms
image 3/40 /content/drive/MyDrive/dataset/images/val/Copy of redwood_image_05_aug4_aug2.jpg: 256x416 125 redwoods, 298.5ms
image 4/40 /content/drive/MyDrive/dataset/images/val/Copy of redwood_image_05_aug4_aug3.jpg: 256x416 97 redwoods, 282.9ms
image 5/40 /content/drive/MyDrive/dataset/images/val/Copy of redwood_image_05_aug4_aug4.jpg: 256x416 149 redwoods, 282.6ms
image 6/40 /content/drive/MyDrive/dataset/images/val/redwood_area_05.jpg: 224x416 1 redwood, 259.3ms
image 7/40 /content/drive/MyDrive/dataset/images/val/redwood_area_06.jpg: 224x416 2 redwoods, 249.1ms
image 8/40 /content/drive/MyDrive/dataset/images/val/redwood_image_01.jpg: 256x416 89 redwoods, 284.4ms
image 9/40 /content/drive/MyDrive/dataset/images/val/redwood_image_02.jpg: 192x416 3

In [ ]:
model.export(format="onnx")

Ultralytics 8.3.200 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/content/redwood_detection/yolov8n_redwood3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.67', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 3.2s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.0 opset 19...
ONNX: slimming with onnxslim 0.1.68...
ONNX: export success ✅ 5.5s, saved as '/content/redwood_detection/yolov8n_redwood3/weights/best.onnx' (11.7 MB)

Export complete (6.0s)
Results saved to /content/redwood_detection/yolov8n_redwood3/weights
Predict:         yolo predict task=detect model=/conten

'/content/redwood_detection/yolov8n_redwood3/weights/best.onnx'